In [ ]:
import os
import tensorflow.compat.v1 as tf
from glob import glob
from PIL import Image
import gc

from model import lowlight_enhance
from utils import *

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

def lowlight_test(test_dir, save_dir, decom_flag=0):
    if not os.path.exists(test_dir):
        print(f"[!] Test directory {test_dir} bestaat niet")
        return

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    test_low_data_name = glob(os.path.join(test_dir, '*.*'))
    test_low_data = []

    for img_path in test_low_data_name:
        img = load_images(img_path)
        test_low_data.append(img)

    tf.reset_default_graph()
    gc.collect()

    try:
        with tf.Session() as sess:
            model = lowlight_enhance(sess)
            model.test(test_low_data, [], test_low_data_name, save_dir=save_dir, decom_flag=decom_flag)
    except Exception as e:
        print(f"Fout tijdens verwerking van {test_dir}: {e}")

    # Resources expliciet loslaten
    del test_low_data
    del model
    gc.collect()

    print(f"Testing completed for {test_dir}, results saved in {save_dir}")


def process_rooms(data_root):
    rooms_root = os.path.join(data_root, '../data-filtered-rooms')
    room_folders = [f for f in os.listdir(rooms_root) if os.path.isdir(os.path.join(rooms_root, f))]

    room = "room9"
    room_path = os.path.join(rooms_root, room)
    save_dir = f"../data-retinexnet/{room}"
    print(f"Processing room: {room}")
    lowlight_test(room_path, save_dir, decom_flag=0)

def main():
    data_root = './'
    process_rooms(data_root)

main()
